Base에서 실행

In [1]:
import torch

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import reuters
import keras

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(test_split=0.2)

category_list = []
for i in range(46):
    category_list.append(0)
print(len(category_list))

for i in train_labels:
    category_list[i] += 1

extracted_index = []

for i, v in enumerate(category_list):
    if v >= 100:
#         print(i, v)
        extracted_index.append(i)


extracted_train_data = []
for i, v in enumerate(train_labels):
    if v in extracted_index:
        extracted_train_data.append(train_data[i])

extracted_test_data = []
for i, v in enumerate(test_labels):
    if v in extracted_index:
        extracted_test_data.append(test_data[i])

extracted_train_labels = []
for i, v in enumerate(train_labels):
    if v in extracted_index:
        extracted_train_labels.append(train_labels[i])

extracted_test_labels = []
for i, v in enumerate(test_labels):
    if v in extracted_index:
        extracted_test_labels.append(test_labels[i])

word_to_index = reuters.get_word_index()

index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value+3] = key # pad, sos, unk, NULL 포함

for index, token in enumerate(("<pad>", "<sos>", "<unk>")): # index=3은 NULL
    index_to_word[index] = token

#문장 추출
to_txt=[]
for i in range(7828):
  
    to_txt.append(' '.join([index_to_word[index] for index in extracted_train_data[i]])[6:])
    
for i in range(1934):
    to_txt.append(' '.join([index_to_word[index] for index in extracted_test_data[i]])[6:])
    
y=extracted_train_labels+extracted_test_labels

encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=label[:7828]
y_test=label[7828:]    


x_train=to_txt[:7828]
x_test=to_txt[7828:]


sos_x_train=[]
sos_x_test=[]
for sen in x_train:
    sos_x_train.append('<sos>'+sen)
for sen in x_test:
    sos_x_test.append('<sos>'+sen)

all_txt=sos_x_train+sos_x_test

from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_txt)

vocab_size =len(tokenizer.word_index)

x_train_encoded = tokenizer.texts_to_sequences(sos_x_train)
x_test_encoded = tokenizer.texts_to_sequences(sos_x_test)

max_len = 300

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_encoded, maxlen=max_len)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_encoded, maxlen=max_len)

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs


class TransformerBlock(tf.keras.layers.Layer):
    
        
    def __init__(self, embedding_dim, num_heads, dff, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.dff = dff
        self.rate = rate
        self.att = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(dff, activation="relu"),
             tf.keras.layers.Dense(embedding_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs) # 첫번째 서브층 : 멀티 헤드 어텐션
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output) # Add & Norm
        ffn_output = self.ffn(out1) # 두번째 서브층 : 포지션 와이즈 피드 포워드 신경망
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output) # Add & Norm
    
    def get_config(self):
        config = super(TransformerBlock, self).get_config()
        config.update({
            'embedding_dim': self.embedding_dim,
            'num_heads': self.num_heads,
            'dff': self.dff,
            'rate': self.rate
        })
        return config


class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embedding_dim, **kwargs):
        super(TokenAndPositionEmbedding, self).__init__(**kwargs)
    
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=max_len, output_dim=embedding_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
    def get_config(self):
        config = super(TokenAndPositionEmbedding, self).get_config()
        config.update({
            'max_len': self.max_len,
            'vocab_size': self.vocab_size,
            'embedding_dim': self.embedding_dim
        })
        return config


embedding_dim = 1024  # 각 단어의 임베딩 벡터의 차원
num_heads = 1  # 어텐션 헤드의 수
dff = 1024  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기
num_transformer_blocks = 1

inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)

for _ in range(num_transformer_blocks):
    transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
    x = transformer_block(x)

# transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
# x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(12, activation="softmax")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.summary()

y_train = np.array(y_train)
y_test = np.array(y_test)

46
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300)]             0         
                                                                 
 token_and_position_embeddin  (None, 300, 1024)        29426688  
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_block (Transfor  (None, 300, 1024)        6301696   
 merBlock)                                                       
                                                                 
 global_average_pooling1d (G  (None, 1024)             0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0  

In [2]:
x_val=x_train[:1000]
x_train=x_train[1000:]
y_val=y_train[:1000]
y_train=y_train[1000:]

In [3]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, verbose=1),
]

In [4]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(x_train, y_train,callbacks=callbacks, batch_size=256, epochs=15, validation_data=(x_val, y_val))

print("테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))

Epoch 1/15
27/27 [==============================] - 8s 235ms/step - loss: 2.7534 - accuracy: 0.3366 - val_loss: 1.6501 - val_accuracy: 0.4780 - lr: 0.0010
Epoch 2/15
27/27 [==============================] - 6s 234ms/step - loss: 1.4035 - accuracy: 0.5426 - val_loss: 1.1018 - val_accuracy: 0.6500 - lr: 0.0010
Epoch 3/15
27/27 [==============================] - 6s 233ms/step - loss: 1.0523 - accuracy: 0.6670 - val_loss: 1.0397 - val_accuracy: 0.6930 - lr: 0.0010
Epoch 4/15
27/27 [==============================] - 6s 232ms/step - loss: 0.8546 - accuracy: 0.7378 - val_loss: 0.8502 - val_accuracy: 0.7650 - lr: 0.0010
Epoch 5/15
27/27 [==============================] - 6s 233ms/step - loss: 0.6339 - accuracy: 0.8188 - val_loss: 0.7429 - val_accuracy: 0.7910 - lr: 0.0010
Epoch 6/15
27/27 [==============================] - 6s 233ms/step - loss: 0.4671 - accuracy: 0.8711 - val_loss: 0.7220 - val_accuracy: 0.8280 - lr: 0.0010
Epoch 7/15
27/27 [==============================] - 6s 236ms/step - lo

In [5]:
from keras.models import load_model
custom_objects = {"TokenAndPositionEmbedding": TokenAndPositionEmbedding, "TransformerBlock": TransformerBlock}
model = load_model('best_model.h5', custom_objects=custom_objects)
model.evaluate(x_test, y_test)

61/61 [==============================] - 1s 11ms/step - loss: 0.7236 - accuracy: 0.8325


[0.7235599756240845, 0.8324715495109558]

In [7]:
np.average([83.5,83.6,83.4])

83.5